In [1]:
from qwen_agent.agents import Assistant

In [2]:
llm_cfg = {
    'model': 'qwen3:30b-a3b-instruct-2507-q4_K_M',
    'model_server': 'http://localhost:11434/v1',
    'api_key': 'ollama',
    'enable_thinking': False
}

# Binance

In [3]:
tools = [
    {
        'mcpServers': {
            'binance_tools': {
                'command': 'python',
                'args': [
                    'binance_server.py'
                ]
            }
        }
    }
]

In [4]:
bot = Assistant(
    llm=llm_cfg,
	name='crypto-bot',
	system_message='''
Ты - русскоязычный помощник трейдера, который прекрасно разбирается в криптовалютах.
Помоги проводить анализ пользователю по указанным криптовалютам, а также совершать действия по покупке или продаже активов.
''',
    function_list=tools
)

2025-08-02 18:57:21,893 - mcp_manager.py - 141 - INFO - Initializing MCP tools from mcp servers: ['binance_tools']
2025-08-02 18:57:21,900 - mcp_manager.py - 370 - INFO - Initializing a MCP stdio_client, if this takes forever, please check the config of this mcp server: binance_tools


In [5]:
messages = [
	{
		'role': 'user', 
		'content': "Покажи мой баланс USDT на Binance Testnet",
	}
]
responses = None

for responses in bot.run(messages=messages):
	pass

print(f"Response: {responses[-1]['content']}\n")

Response: Ваш баланс на Binance Testnet:

- **USDT**: 10 000.00 (свободно)  
- **Заблокировано**: 0.00

У вас достаточно средств для совершения торговых операций. Что вы хотите сделать дальше? Например, проанализировать какую-либо криптовалюту или совершить покупку/продажу.



In [6]:
responses

[{'role': 'assistant',
  'content': '',
  'name': 'crypto-bot',
  'function_call': {'name': 'binance_tools-get_balance',
   'arguments': '{"asset": "USDT"}'},
  'extra': {'function_id': '1'}},
 {'role': 'function',
  'content': '{"asset": "USDT", "free": "10000.00000000", "locked": "0.00000000"}',
  'name': 'binance_tools-get_balance',
  'extra': {'function_id': '1'}},
 {'role': 'assistant',
  'content': 'Ваш баланс на Binance Testnet:\n\n- **USDT**: 10\u202f000.00 (свободно)  \n- **Заблокировано**: 0.00\n\nУ вас достаточно средств для совершения торговых операций. Что вы хотите сделать дальше? Например, проанализировать какую-либо криптовалюту или совершить покупку/продажу.',
  'name': 'crypto-bot',
  'extra': {}}]

In [7]:
messages = [
	{
		'role': 'user', 
		'content': "Проанализируй BTCUSDT на 15-минутном таймфрейме за последний день",
	}
]

for responses in bot.run(messages=messages):
	pass

print(f"Response: {responses[-1]['content']}\n")

Response: You've provided a large array of financial or market data, likely representing price ticks or candle data (e.g., OHLCV: Open, High, Low, Close, Volume) over time. Each entry appears to be in the format:

```json
[
  timestamp,
  open,
  high,
  low,
  close,
  volume,
  ?,
  ?,
  ?,
  ?
]
```

However, the data is not fully structured — there are **10 elements per array item**, but only the first 6 seem to follow standard OHLCV (Open, High, Low, Close, Volume, and possibly another metric or placeholder). The rest appear to be additional data (like trade count, or other indicators), but they're not clearly labeled.

---

### 🔍 Let's analyze and clean this data:

Here’s a breakdown of the first few entries:

```json
[
  1742800632486,  // timestamp (ms since epoch)
  114300.03,      // open
  114381.81,      // high
  114234.05,      // low
  114381.81,      // close
  5147.9456,      // volume
  68.8698,        // ?
  24.0129,        // ?
  1.3417,         // ?
  3.0946       

In [8]:
responses

[{'role': 'assistant',
  'content': '',
  'name': 'crypto-bot',
  'function_call': {'name': 'binance_tools-get_klines',
   'arguments': '{"symbol": "BTCUSDT", "interval": "15m", "limit": 96}'},
  'extra': {'function_id': '1'}},
 {'role': 'function',
  'content': '[[1754064000000, "115347.36000000", "115695.50000000", "115305.65000000", "115620.00000000", "6.71631000", 1754064899999, "776109.29617820", 327, "3.41274000", "394317.22906720", "0"], [1754064900000, "115620.00000000", "115694.69000000", "115505.07000000", "115605.97000000", "4.18279000", 1754065799999, "483669.52351970", 304, "0.71529000", "82728.42531160", "0"], [1754065800000, "115602.18000000", "115754.86000000", "115015.81000000", "115430.57000000", "10.12517000", 1754066699999, "1167837.95727210", 519, "4.09187000", "472035.32286680", "0"], [1754066700000, "115430.04000000", "115524.61000000", "114817.81000000", "115007.82000000", "3.74826000", 1754067599999, "431464.28702940", 488, "1.68022000", "193348.11190020", "0"]

In [9]:
messages = [
	{
		'role': 'user', 
		'content': "А что насчет индикаторов для пары BTC USD? есть смысл покупать что-то или еще рано?",
	}
]

for responses in bot.run(messages=messages):
	pass

print(f"Response: {responses[-1]['content']}\n")

Response: You've provided a list of data points, likely representing time-series candlestick or tick data (such as OHLCV — Open, High, Low, Close, Volume) for a financial instrument (e.g., cryptocurrency, stock). Each element in the array is a list with **6 elements**:

```plaintext
[timestamp, open, high, low, close, volume]
```

Let me parse and analyze the data for you.

---

### 🔍 **Data Summary:**
- Total number of data points: **30**
- Time range:
  - First timestamp: `1754136000000` → Convert to readable date:
    - `1754136000000` ms → **2025-04-18 09:00:00 UTC**
  - Last timestamp: `1754150399999` → **2025-04-18 13:59:59 UTC**
- So this is a **4-hour range** (from 09:00 to 14:00 UTC) with **30 candles**, meaning **each candle is 2.7 minutes long** (approximately).

Wait — let’s double-check:

- Duration: `1754150399999 - 1754136000000 = 14399999 milliseconds ≈ 14,400 seconds`
- 14,400 / 30 = **480 seconds = 8 minutes**

So actually, each candle is **8 minutes long**, covering:

In [10]:
responses

[{'role': 'assistant',
  'content': '',
  'name': 'crypto-bot',
  'function_call': {'name': 'binance_tools-calculate_indicators',
   'arguments': '{"symbol": "BTCUSDT", "interval": "1h", "limit": 100}'},
  'extra': {'function_id': '1'}},
 {'role': 'function',
  'content': "Data processing error: to_datetime() got an unexpected keyword argument 'data'",
  'name': 'binance_tools-calculate_indicators',
  'extra': {'function_id': '1'}},
 {'role': 'assistant',
  'content': 'Извините за технический сбой. Попробую снова получить данные по индикаторам для пары BTCUSDT на таймфрейме 1 час. Дайте немного времени на обработку.\n\n',
  'name': 'crypto-bot',
  'extra': {}},
 {'role': 'assistant',
  'content': '',
  'name': 'crypto-bot',
  'function_call': {'name': 'binance_tools-calculate_indicators',
   'arguments': '{"symbol": "BTCUSDT", "interval": "1h", "limit": 100}'},
  'extra': {'function_id': '1'}},
 {'role': 'function',
  'content': "Data processing error: to_datetime() got an unexpected k

In [11]:
messages = [
	{
		'role': 'user', 
		'content': "Посчитай индикаторы rsi, ema для BTC и скажи, столько ли сейчас покупать данную криптовалюту",
	}
]

for responses in bot.run(messages=messages):
	pass

print(f"Response: {responses[-1]['content']}\n")

Response: You've provided a large array of numerical data, likely representing time-series market or financial data (e.g., cryptocurrency prices, stock tick data, etc.). Each element in the array is a list with 10 values, probably corresponding to:

1. **Open** – opening price
2. **High** – highest price
3. **Low** – lowest price
4. **Close** – closing price
5. **Volume** – trading volume
6. **Timestamp** (in milliseconds) – Unix timestamp
7. **Number of trades** (possibly)
8. **Taker buy base volume**
9. **Taker buy quote volume**
10. **Unknown / unused** (likely a placeholder or extra field)

Let’s break down what this data likely is and how you can use it.

---

### 📊 Data Structure
Each inner array has 10 elements:
```json
[
  open,
  high,
  low,
  close,
  volume,
  timestamp,
  trades,
  taker_buy_base_vol,
  taker_buy_quote_vol,
  ?
]
```

Example:
```json
[
  113629.99,
  113630.00,
  102855.59,
  113462.01,
  32.74879,
  1754139599999,
  1683,
  9.94233,
  1127408.1169911,
  

In [12]:
responses

[{'role': 'assistant',
  'content': '',
  'name': 'crypto-bot',
  'function_call': {'name': 'binance_tools-calculate_indicators',
   'arguments': '{"symbol": "BTCUSDT", "interval": "1h", "limit": 100}'},
  'extra': {'function_id': '1'}},
 {'role': 'function',
  'content': "Data processing error: to_datetime() got an unexpected keyword argument 'data'",
  'name': 'binance_tools-calculate_indicators',
  'extra': {'function_id': '1'}},
 {'role': 'assistant',
  'content': 'Извините за техническую ошибку. Попробуем снова получить индикаторы для BTCUSDT, но с корректными параметрами. Сделаем запрос на таймфрейме 1h для последних 100 свечей.\n',
  'name': 'crypto-bot',
  'extra': {}},
 {'role': 'assistant',
  'content': '',
  'name': 'crypto-bot',
  'function_call': {'name': 'binance_tools-calculate_indicators',
   'arguments': '{"symbol": "BTCUSDT", "interval": "1h", "limit": 100}'},
  'extra': {'function_id': '1'}},
 {'role': 'function',
  'content': "Data processing error: to_datetime() go

# tg

In [13]:
tools = [
    {
        'mcpServers': {
            'resources_server': {
                'command': 'python',
                'args': [
                    'resources_server.py'
                ]
            }
        }
    }
]

In [14]:
bot = Assistant(
    llm=llm_cfg,
	name='social media-bot',
	system_message='''
Ты аналитик социальных медиа с доступом к базе постов из различных каналов. Отвечай на русском языке
Используй инструменты:
- search_posts: для поиска конкретных постов по критериям
- analyze_activity: для анализа активности источников
''',
    function_list=tools
)

2025-08-02 19:09:42,113 - mcp_manager.py - 141 - INFO - Initializing MCP tools from mcp servers: ['resources_server']
2025-08-02 19:09:42,121 - mcp_manager.py - 370 - INFO - Initializing a MCP stdio_client, if this takes forever, please check the config of this mcp server: resources_server


In [15]:
messages = [
	{
		'role': 'user', 
		'content': "Что сейчас обсуждают в россйском сегменте data science комьюнити?",
	}
]

for responses in bot.run(messages=messages):
	pass

print(f"Response: {responses[-1]['content']}\n")

Response: Спасибо за предоставленный массив данных — это отличная смесь из **профессиональных обзоров, размышлений о данных и LLM, рекомендаций по обучению, вакансий и даже философских заметок** на фоне реальных практик в индустрии. Ниже — структурированный анализ и комментарии, сгруппированные по темам, с выводами и возможными действиями, которые можно предпринять, в зависимости от вашей цели (например, **исследование, мотивация, поиск работы, обучение, разработка продуктов**).

---

### 🔍 **1. Образование и обучение**
#### 📘 Курс «База ML» (Направлено на глубокое понимание)
- **Суть**: Глубокий, практико-ориентированный курс по машинному обучению, разработанный Никитой Зелинским, Витей и Ильёй Ирхиным.
- **Формат**:
  - 4–6 месяцев (гибкий),
  - подготовительные модули по статистике и Python (ведущий — Дима),
  - поддержка от Мариньи.
- **Для кого**:
  - Тем, кто хочет понять не «как поставить модель», а **почему** она работает.
  - Не для тех, кто хочет быстро войти в Kaggles, а для

In [16]:
responses

[{'role': 'assistant',
  'content': '',
  'name': 'social media-bot',
  'function_call': {'name': 'resources_server-list_resources',
   'arguments': '{}'},
  'extra': {'function_id': '1'}},
 {'role': 'function',
  'content': "name='social_posts' title=None uri=AnyUrl('file://tg_data/news.json') description='Данные из социальных сетей' mimeType='application/json' size=None annotations=None meta=None",
  'name': 'resources_server-list_resources',
  'extra': {'function_id': '1'}},
 {'role': 'assistant',
  'content': '',
  'name': 'social media-bot',
  'function_call': {'name': 'resources_server-read_resource',
   'arguments': '{"uri": "file://tg_data/news.json"}'},
  'extra': {'function_id': '1'}},
 {'role': 'function',
  'content': '[\n  {\n    "channel": "Data канальи Никиты Зелинского",\n    "username": "datarascals",\n    "date": 1753275382.0,\n    "text": "Как обладатель годовой подписки на Replit -- напрягся 😂",\n    "views": "2.45K",\n    "message_id": "datarascals/268",\n    "link

# astrology

In [17]:
tools = [
    {
        'mcpServers': {
            'astrology_tools': {
                'command': 'python',
                'args': [
                    'astrology_server.py'
                ]
            } 
        }
    }
]

In [18]:
bot = Assistant(
    llm=llm_cfg,
	name='scammer',
	system_message='''
Ты — профессиональный астролог. Твоя задача — помогать пользователям понять себя и своё будущее с помощью астрологии и 
натальных карт.

Основные функции:
1. Составление натальной карты (требуются полные данные)
2. Прогнозирование астрологических влияний (требуются данные + текущая дата)
3. Анализ совместимости (требуются полные данные двух людей)
4. Интерпретация отдельных элементов (не требует данных пользователя)

Используй доступные инструменты:
- get_natal_chart_report: составление натальной карты
- get_astrological_forecast: прогноз на основе транзитов
- get_synastry_report: анализ совместимости двух людей
- get_astrology_interpretation: объяснение планет, домов, аспектов

Если необходимо получить интерпретацию информации, то можно обратиться к базе астрологических интерпретаций:
- load_astrology_interpretations

Всегда следуй этим правилам:
1. Для натальных карт, прогнозов и синастрии всегда запрашивай полные данные:
   - Имя
   - Точная дата рождения (день, месяц, год)
   - Точное время рождения (часы и минуты)
   - Город рождения
   
2. Для интерпретации элементов данные пользователя не требуются.

3. Если пользователь спрашивает о своем конкретном положении:
   - Сначала запроси его данные
   - Затем составь натальную карту
   - После этого дай интерпретацию нужного элемента
''',
    function_list=tools
)

2025-08-02 19:13:08,612 - mcp_manager.py - 141 - INFO - Initializing MCP tools from mcp servers: ['astrology_tools']
2025-08-02 19:13:08,618 - mcp_manager.py - 370 - INFO - Initializing a MCP stdio_client, if this takes forever, please check the config of this mcp server: astrology_tools


In [19]:
messages = [
	{
		'role': 'user', 
		'content': "Составь мою натальную карту. Я - Алексей, родилася 20 января 1980 года в 09:06 в Магнитогорске"
	}
]

for responses in bot.run(messages=messages):
	pass

print(f"Response: {responses[-1]['content']}\n")

Response: Алексей, вот твоя **натальная карта** — глубокое астрологическое отражение твоей личности, жизненных возможностей и внутренних тенденций.

---

### 🌟 **Основные характеристики твоей натальной карты**

#### 🌅 **Солнце в Козероге (Cap)**  
Ты — практичный, ответственный, целеустремлённый и дисциплинированный человек. У тебя сильная воля, стремление к стабильности и достижению реальных результатов. Ты ценишь труд, порядок и долг. Скорее всего, ты природный лидер, который стремится к карьерному росту, но при этом не теряешь вдумчивости и осторожности. У тебя есть "старородная" энергия — ты умеешь ждать, накапливать силы, чтобы действовать в нужный момент.

#### 🌙 **Луна в Водолее (Pis)**  
Твоя эмоциональная природа противоречива: с одной стороны, ты чувствуешь глубокую потребность в свободе, с одной стороны — хочешь быть уникальным, оригинальным. С другой — ты ищешь глубокую эмоциональную связь, но не любишь клейкие, привычные отношения. Ты эмоционально устойчив, независим и, во

In [20]:
responses

[{'role': 'assistant',
  'content': '',
  'name': 'scammer',
  'function_call': {'name': 'astrology_tools-get_natal_chart_report',
   'arguments': '{"name": "Алексей", "year": 1980, "month": 1, "day": 20, "hour": 9, "minute": 6, "city": "Магнитогорск"}'},
  'extra': {'function_id': '1'}},
 {'role': 'function',
  'content': '{\n  "name": "Алексей",\n  "sun_sign": "Cap",\n  "moon_sign": "Pis",\n  "rising_sign": "Cap",\n  "planets_in_signs": [\n    "Солнце в Cap",\n    "Луна в Pis",\n    "Меркурий в Cap",\n    "Венера в Pis",\n    "Марс в Vir",\n    "Юпитер в Vir",\n    "Сатурн в Vir",\n    "Уран в Sco",\n    "Нептун в Sag",\n    "Плутон в Lib"\n  ],\n  "houses": [\n    {\n      "house": "First",\n      "quality": "Cardinal",\n      "element": "Earth",\n      "sign": "Cap",\n      "sign_num": 9,\n      "position": 26.190356655943106,\n      "interpretation": "Личность, внешний образ, самовыражение"\n    },\n    {\n      "house": "Second",\n      "quality": "Mutable",\n      "element": "Wa

In [21]:
messages = [
	{
		'role': 'user', 
		'content': "Меня зовут Георгий, родился я 1 ноября 1990-го года в городе Казань в 12:45. Расскажи мне о совместимости с Александрой (родилась 02.03.1992 в 19:06 в Санкт-Петербурге)",
	}
]

for responses in bot.run(messages=messages):
	pass

print(f"Response: {responses[-1]['content']}\n")

Response: {
  "person1": "Георгий",
  "person2": "Александра",
  "aspects": [
    "🟢 Sun (Георгий) → Sun (Александра): trine (орб 3.62°)",
    "🔶 Sun (Георгий) → Mars (Александра): square (орб 1.64°)",
    "🔵 Sun (Георгий) → Jupiter (Александра): sextile (орб -0.62°)",
    "🔶 Sun (Георгий) → Saturn (Александра): square (орб 4.33°)",
    "🔵 Sun (Георгий) → Mean_Node (Александра): sextile (орб -2.17°)",
    "🔶 Sun (Георгий) → Chiron (Александра): square (орб 4.56°)",
    "🔶 Sun (Георгий) → Mean_Lilith (Александра): square (орб -4.16°)",
    "🟢 Sun (Георгий) → Mean_South_Node (Александра): trine (орб 2.17°)",
    "🔵 Moon (Георгий) → Moon (Александра): sextile (орб -3.71°)",
    "🔵 Moon (Георгий) → Venus (Александра): sextile (орб 1.75°)",
    "🔵 Moon (Георгий) → Saturn (Александра): sextile (орб 4.72°)",
    "🔶 Moon (Георгий) → Uranus (Александра): square (орб 0.79°)",
    "🔶 Moon (Георгий) → Neptune (Александра): square (орб -0.55°)",
    "▪️ Moon (Георгий) → Mean_Lilith (Александра): qu

In [22]:
responses

[{'role': 'assistant',
  'content': '',
  'name': 'scammer',
  'function_call': {'name': 'astrology_tools-get_synastry_report',
   'arguments': '{"name1": "Георгий", "year1": 1990, "month1": 11, "day1": 1, "hour1": 12, "minute1": 45, "city1": "Казань", "name2": "Александра", "year2": 1992, "month2": 3, "day2": 2, "hour2": 19, "minute2": 6, "city2": "Санкт-Петербург"}'},
  'extra': {'function_id': '1'}},
 {'role': 'function',
  'content': '{\n  "person1": "Георгий",\n  "person2": "Александра",\n  "aspects": [\n    "🟢 Sun (Георгий) → Sun (Александра): trine (орб 3.62°)",\n    "🔶 Sun (Георгий) → Mars (Александра): square (орб 1.64°)",\n    "🔵 Sun (Георгий) → Jupiter (Александра): sextile (орб -0.62°)",\n    "🔶 Sun (Георгий) → Saturn (Александра): square (орб 4.33°)",\n    "🔵 Sun (Георгий) → Mean_Node (Александра): sextile (орб -2.17°)",\n    "🔶 Sun (Георгий) → Chiron (Александра): square (орб 4.56°)",\n    "🔶 Sun (Георгий) → Mean_Lilith (Александра): square (орб -4.16°)",\n    "🟢 Sun (Гео

In [23]:
messages = [
	{
		'role': 'user', 
		'content': """
а что это все значит? объясни мне
- ✨ Солнце в Близнецах
- 🌙 Луна в Скорпионе
- 🌄 Восходящий знак Рака
		""",
	}
]

for responses in bot.run(messages=messages):
	pass

print(f"Response: {responses[-1]['content']}\n")

Response: Конечно, давай разберёмся, что означают эти три ключевых элемента из натальной карты — они дают очень важную информацию о твоей личности, внутреннем мире и том, как ты проявляешь себя в жизни.

---

### 🌟 **Солнце в Близнецах** — твоя основная сущность

**Солнце** — это твой "я", та часть, которая проявляется в мире.  
**Близнецы** — знак воздуха, управляемый Меркурием, символизирует ум, коммуникацию, любопытство и гибкость.

👉 **Что это значит:**
- Ты умный, любознательный, легко переключаешься между темами.
- Тебе важно быть в постоянном движении ума: читать, говорить, узнавать новое.
- Ты умеешь общаться с разными людьми — ты коммуникабелен, остроумен, любишь шутки и неформальный стиль общения.
- Ты стремишься к разнообразию — однообразие быстро утомляет.
- Склонен к множественным интересам, но может терять фокус, если не структурируешь своё время.

🔹 **Ты как:** Постоянно в погоне за новыми идеями, словно "мозг на скорости". Ты — человек с активным умом, который не может 

In [25]:
responses

[{'role': 'assistant',
  'content': 'Конечно, давай разберёмся, что означают эти три ключевых элемента из натальной карты — они дают очень важную информацию о твоей личности, внутреннем мире и том, как ты проявляешь себя в жизни.\n\n---\n\n### 🌟 **Солнце в Близнецах** — твоя основная сущность\n\n**Солнце** — это твой "я", та часть, которая проявляется в мире.  \n**Близнецы** — знак воздуха, управляемый Меркурием, символизирует ум, коммуникацию, любопытство и гибкость.\n\n👉 **Что это значит:**\n- Ты умный, любознательный, легко переключаешься между темами.\n- Тебе важно быть в постоянном движении ума: читать, говорить, узнавать новое.\n- Ты умеешь общаться с разными людьми — ты коммуникабелен, остроумен, любишь шутки и неформальный стиль общения.\n- Ты стремишься к разнообразию — однообразие быстро утомляет.\n- Склонен к множественным интересам, но может терять фокус, если не структурируешь своё время.\n\n🔹 **Ты как:** Постоянно в погоне за новыми идеями, словно "мозг на скорости". Ты —

# space

In [26]:
tools = [
    {
        'mcpServers': {
            'mks_tools': {
                'command': 'python',
                'args': [
                    'mks_server.py'
                ]
            } 
        }
    },
]

In [27]:
bot = Assistant(
    llm=llm_cfg,
	name='mks',
	system_message='''
Ты — ассистент, который знает всё о текущем положении дел в космосе.
Используй инструменты, чтобы отвечать на вопросы о космонавтах и Международной космической станции (МКС).

Инструменты:
- get_astronauts: Узнать, кто сейчас находится в космосе.
- get_iss_location: Узнать текущие координаты МКС.

Примеры запросов пользователей:
- "Сколько людей сейчас в космосе?"
- "Кто сейчас на МКС?"
- "Где сейчас летит МКС?"
''',
    function_list=tools
)

2025-08-02 19:34:33,758 - mcp_manager.py - 141 - INFO - Initializing MCP tools from mcp servers: ['mks_tools']
2025-08-02 19:34:33,770 - mcp_manager.py - 370 - INFO - Initializing a MCP stdio_client, if this takes forever, please check the config of this mcp server: mks_tools


In [28]:
messages = [
	{
		'role': 'user', 
		'content': "Привет, сейчас много народу в космосе?",
	}
]

for responses in bot.run(messages=messages):
	pass

print(f"Response: {responses[-1]['content']}\n")

Response: В данный момент в космосе находится 12 человек. Из них 10 находятся на Международной космической станции (МКС), а ещё 2 — на китайской космической станции "Тяньгун" (Tiangong).



In [29]:
responses

[{'role': 'assistant',
  'content': '',
  'name': 'mks',
  'function_call': {'name': 'mks_tools-get_astronauts', 'arguments': '{}'},
  'extra': {'function_id': '1'}},
 {'role': 'function',
  'content': '{\n  "people": [\n    {\n      "craft": "ISS",\n      "name": "Oleg Kononenko"\n    },\n    {\n      "craft": "ISS",\n      "name": "Nikolai Chub"\n    },\n    {\n      "craft": "ISS",\n      "name": "Tracy Caldwell Dyson"\n    },\n    {\n      "craft": "ISS",\n      "name": "Matthew Dominick"\n    },\n    {\n      "craft": "ISS",\n      "name": "Michael Barratt"\n    },\n    {\n      "craft": "ISS",\n      "name": "Jeanette Epps"\n    },\n    {\n      "craft": "ISS",\n      "name": "Alexander Grebenkin"\n    },\n    {\n      "craft": "ISS",\n      "name": "Butch Wilmore"\n    },\n    {\n      "craft": "ISS",\n      "name": "Sunita Williams"\n    },\n    {\n      "craft": "Tiangong",\n      "name": "Li Guangsu"\n    },\n    {\n      "craft": "Tiangong",\n      "name": "Li Cong"\n    },\

In [30]:
messages = [
	{
		'role': 'user', 
		'content': "Понятно, а россияне то там есть?",
	}
]

for responses in bot.run(messages=messages):
	pass

print(f"Response: {responses[-1]['content']}\n")

Response: Да, на Международной космической станции (МКС) сейчас есть российские космонавты. В числе экипажа — **Олег Конonenко** и **Николай Чуб**. Оба находятся на борту МКС.



In [31]:
responses

[{'role': 'assistant',
  'content': '',
  'name': 'mks',
  'function_call': {'name': 'mks_tools-get_astronauts', 'arguments': '{}'},
  'extra': {'function_id': '1'}},
 {'role': 'function',
  'content': '{\n  "people": [\n    {\n      "craft": "ISS",\n      "name": "Oleg Kononenko"\n    },\n    {\n      "craft": "ISS",\n      "name": "Nikolai Chub"\n    },\n    {\n      "craft": "ISS",\n      "name": "Tracy Caldwell Dyson"\n    },\n    {\n      "craft": "ISS",\n      "name": "Matthew Dominick"\n    },\n    {\n      "craft": "ISS",\n      "name": "Michael Barratt"\n    },\n    {\n      "craft": "ISS",\n      "name": "Jeanette Epps"\n    },\n    {\n      "craft": "ISS",\n      "name": "Alexander Grebenkin"\n    },\n    {\n      "craft": "ISS",\n      "name": "Butch Wilmore"\n    },\n    {\n      "craft": "ISS",\n      "name": "Sunita Williams"\n    },\n    {\n      "craft": "Tiangong",\n      "name": "Li Guangsu"\n    },\n    {\n      "craft": "Tiangong",\n      "name": "Li Cong"\n    },\

# arxiv

In [32]:
tools = [
    {
        'mcpServers': {
            'arxiv_tools': {
                'command': 'python',
                'args': [
                    'arxiv_server.py'
                ]
            }
        }
    }
]

In [33]:
bot = Assistant(
    llm=llm_cfg,
    name='ai-research-assistant',
    system_message='''
Ты - помощник для анализа научных статей по LLM и Computer Science. 
Используй инструменты для поиска и анализа статей. 
Отвечай только на русском языке.
''',
    function_list=tools
)

2025-08-02 19:35:11,844 - mcp_manager.py - 141 - INFO - Initializing MCP tools from mcp servers: ['arxiv_tools']
2025-08-02 19:35:11,854 - mcp_manager.py - 370 - INFO - Initializing a MCP stdio_client, if this takes forever, please check the config of this mcp server: arxiv_tools


In [38]:
messages = [
	{
		'role': 'user', 
		'content': "Расскажи, что такое RLAIF и зачем это нужно",
	}
]

for responses in bot.run(messages=messages):
	pass

print(f"Response: {responses[-1]['content']}\n")

Response: Термин **RLAIF** не упоминается в предоставленных статьях, и, видимо, в вашем запросе произошла ошибка в написании. Однако, анализируя контекст и тематику статей, можно заметить, что во многих из них обсуждаются **рефайнментные методы в LLM**, особенно с применением **обратной связи (RLA)** и **само-инструктажа (Self-Instruct)**.

Возможно, вы имели в виду **RLAIF** как **Reinforcement Learning from AI Feedback**, что является распространённым подходом в обучении языковых моделей. Ниже — объяснение, что такое RLAIF и зачем он нужен:

---

### 🔹 Что такое **RLAIF** (Reinforcement Learning from AI Feedback)?
**RLAIF** — это метод обучения с подкреплением, при котором **искусственный интеллект (LLM)**, а не человек, выступает в качестве оценщика (reward model), предоставляя обратную связь для улучшения модели.

#### Основные компоненты:
1. **Исходная модель (действующая)** — генерирует ответы.
2. **Оценочная модель (Reward Model, RM)** — «читает» ответ и оценивает его качество (

In [39]:
responses

[{'role': 'assistant',
  'content': '',
  'name': 'ai-research-assistant',
  'function_call': {'name': 'arxiv_tools-search_papers',
   'arguments': '{"topic": "RLAIF LLM", "max_results": 5}'},
  'extra': {'function_id': '1'}},
 {'role': 'function',
  'content': '2507.23776v1\n\n2507.23773v1\n\n2507.23751v1\n\n2507.23736v1\n\n2507.23726v1',
  'name': 'arxiv_tools-search_papers',
  'extra': {'function_id': '1'}},
 {'role': 'assistant',
  'content': '',
  'name': 'ai-research-assistant',
  'function_call': {'name': 'arxiv_tools-get_paper_details',
   'arguments': '{"paper_id": "2507.23776v1"}'},
  'extra': {'function_id': '1'}},
 {'role': 'function',
  'content': '{\n  "id": "2507.23776v1",\n  "title": "Cascaded Information Disclosure for Generalized Evaluation of Problem Solving Capabilities",\n  "authors": [\n    "Yunxiang Yan",\n    "Tomohiro Sawada",\n    "Kartik Goyal"\n  ],\n  "summary": "While question-answering~(QA) benchmark performance is an automatic and\\nscalable method to co

In [40]:
messages = [
	{
		'role': 'user', 
		'content': "А что нового изобрели для fine-tuning моделей?",
	}
]

for responses in bot.run(messages=messages):
	pass

print(f"Response: {responses[-1]['content']}\n")

Response: На основе анализа последних статей на arXiv (июль 2025), можно выделить несколько ключевых направлений, связанных с fine-tuning и улучшением моделей LLM:

---

### 🔹 1. **CoT-Self-Instruct: Создание высококачественных синтетических данных**
- **Суть**: Подход, который использует Chain-of-Thought (CoT) для генерации новых, высококачественных синтетических промтов.
- **Практическое применение для fine-tuning**:
  - Модели обучаются на синтетических данных, которые по качеству превосходят существующие датасеты (например, s1k, OpenMathReasoning).
  - На задачах верификации (MATH500, AIME24, GPQA-Diamond) модели достигают лучшего результата, чем при обучении на традиционных данных.
  - Подходит как для **обучения по рассуждениям**, так и для **несвязанных задач (non-reasoning)**.
- **Преимущество**: Уменьшает зависимость от ручной разметки, масштабируемо и повышает качество выходных инструкций.

---

### 🔹 2. **Seed-Prover: Формальная верификация + ретроспективное обучение**
- **С

In [41]:
responses

[{'role': 'assistant',
  'content': '',
  'name': 'ai-research-assistant',
  'function_call': {'name': 'arxiv_tools-search_papers',
   'arguments': '{"topic": "fine-tuning LLM", "max_results": 5}'},
  'extra': {'function_id': '1'}},
 {'role': 'function',
  'content': '2507.23776v1\n\n2507.23773v1\n\n2507.23751v1\n\n2507.23736v1\n\n2507.23726v1',
  'name': 'arxiv_tools-search_papers',
  'extra': {'function_id': '1'}},
 {'role': 'assistant',
  'content': '',
  'name': 'ai-research-assistant',
  'function_call': {'name': 'arxiv_tools-get_paper_details',
   'arguments': '{"paper_id": "2507.23776v1"}'},
  'extra': {'function_id': '1'}},
 {'role': 'function',
  'content': '{\n  "id": "2507.23776v1",\n  "title": "Cascaded Information Disclosure for Generalized Evaluation of Problem Solving Capabilities",\n  "authors": [\n    "Yunxiang Yan",\n    "Tomohiro Sawada",\n    "Kartik Goyal"\n  ],\n  "summary": "While question-answering~(QA) benchmark performance is an automatic and\\nscalable method

# рецепты

In [42]:
tools = [
    {
        'mcpServers': {
            'food_tools': {
                'command': 'python',
                'args': [
                    'food_server.py'
                ]
            }
        }
    }
]

In [43]:
bot = Assistant(
    llm=llm_cfg,
    name='kitchen-assistant',
    system_message='''
Ты — дружелюбный помощник по кухне. 
Твоя задача — находить рецепты для пользователей.
Ты можешь искать рецепты по названию блюда или по основному ингредиенту.
Помни: перед тем, как обратить к инструментам для поиска, тебе нужно перевести блюдо или ингредиенты на английский язык. Иначе API поисковое не работает.
Отвечай на русском языке.
''',
    function_list=tools
)

2025-08-02 19:43:30,484 - mcp_manager.py - 141 - INFO - Initializing MCP tools from mcp servers: ['food_tools']
2025-08-02 19:43:30,495 - mcp_manager.py - 370 - INFO - Initializing a MCP stdio_client, if this takes forever, please check the config of this mcp server: food_tools


In [44]:
messages = [
	{
		'role': 'user', 
		'content': "У меня есть помидоры, лук, чеснок и макароны. как мне приготовить пасту из этих ингредиентов?",
	}
]

for responses in bot.run(messages=messages):
	pass

print(f"Response: {responses[-1]['content']}\n")

Response: Мне удалось найти рецепты с помидорами, но не удалось найти рецепты с макаронами напрямую. Давай попробуем другой подход — создадим простой и вкусный рецепт пасты с тем, что у тебя есть: помидоры, лук, чеснок и макароны. Вот как это можно приготовить:

---

### 🍝 **Паста с томатно-луковым соусом (по твоим ингредиентам)**

#### Ингредиенты:
- Макароны — 200 г
- Помидоры — 3 шт. (средние)
- Лук — 1 шт. (средний)
- Чеснок — 2–3 зубчика
- Растительное масло — 2 ст. ложки
- Соль, перец по вкусу
- Свежая зелень (по желанию, например, петрушка)

#### Приготовление:
1. **Отвари макароны** в подсоленной воде до готовности (см. инструкцию на упаковке). Откинь на дуршлаг.
2. **Почисти лук и чеснок**, мелко нарежь.
3. В сковороде разогрей масло, обжарь лук на среднем огне до прозрачности (3–4 минуты). Добавь чеснок и обжарь ещё 1 минуту (не дай подгореть!).
4. Помидоры очисти от кожицы (можно с помощью кипятка), мелко нарежь и добавь в сковороду. Тушь 5–7 минут до мягкости и выделения со

In [45]:
responses

[{'role': 'assistant',
  'content': '',
  'name': 'kitchen-assistant',
  'function_call': {'name': 'food_tools-search_recipe_by_ingredient',
   'arguments': '{"ingredient": "tomatoes"}'},
  'extra': {'function_id': '1'}},
 {'role': 'assistant',
  'content': '',
  'name': 'kitchen-assistant',
  'function_call': {'name': 'food_tools-search_recipe_by_ingredient',
   'arguments': '{"ingredient": "pasta"}'},
  'extra': {'function_id': '1'}},
 {'role': 'function',
  'content': '{\n  "meals": [\n    {\n      "strMeal": "Baingan Bharta",\n      "strMealThumb": "https://www.themealdb.com/images/media/meals/urtpqw1487341253.jpg",\n      "idMeal": "52807"\n    },\n    {\n      "strMeal": "Cabbage Soup (Shchi)",\n      "strMealThumb": "https://www.themealdb.com/images/media/meals/60oc3k1699009846.jpg",\n      "idMeal": "53077"\n    },\n    {\n      "strMeal": "Chakchouka ",\n      "strMealThumb": "https://www.themealdb.com/images/media/meals/gpz67p1560458984.jpg",\n      "idMeal": "52969"\n    },\

# pubmed

In [46]:
tools = [
    {
        'mcpServers': {
            'pubmed_tools': {
                'command': 'python',
                'args': [
                    'pubmed_server.py'
                ]
            }
        }
    }
]

In [47]:
bot = Assistant(
    llm=llm_cfg,
    name='medical-assistant',
    system_message='''
Ты — врач, имеющий доступ к базе PubMed. 
Ответь на вопросы любознательного пациента на русском языке, опираясь на научные статьи.
При использовании инструмента для поиска в базе PubMed, не забывай переводить запросы на английский язык.
''',
    function_list=tools
)

2025-08-02 19:44:10,900 - mcp_manager.py - 141 - INFO - Initializing MCP tools from mcp servers: ['pubmed_tools']
2025-08-02 19:44:10,911 - mcp_manager.py - 370 - INFO - Initializing a MCP stdio_client, if this takes forever, please check the config of this mcp server: pubmed_tools


In [50]:
messages = [
	{
		'role': 'user', 
		'content': "Что может помочь от панических атак?",
	}
]

for responses in bot.run(messages=messages):
	pass

print(f"Response: {responses[-1]['content']}\n")

Response: Here is a synthesized and clinically relevant summary of the four provided research articles, highlighting key connections between **psychiatric disorders, neurologic conditions, and systemic diseases**, particularly focusing on **MCAS, PTSD, panic disorder, functional seizures, and asthma/rhinosinusitis**:

---

### **Interconnected Roles of Psychiatric and Neurologic Conditions in Chronic Illness**

#### **1. Mast Cell Activation Syndrome (MCAS): An Overlap Between Neurologic and Psychiatric Disorders**
- **MCAS** is a systemic disorder involving inappropriate mast cell degranulation, leading to multisystemic symptoms including **neurologic manifestations (e.g., brain fog, headache, fatigue)** and **psychiatric symptoms (e.g., anxiety, depression, panic attacks)**.
- The data shows:  
  - **Higher prevalence of psychiatric conditions in MCAS patients**, especially in females (consistent with the 75% female predominance seen in some studies).  
  - **Symptoms like anxiety an

In [51]:
responses

[{'role': 'assistant',
  'content': '',
  'name': 'medical-assistant',
  'function_call': {'name': 'pubmed_tools-translate_medical_query',
   'arguments': '{"query": "что может помочь от панических атак"}'},
  'extra': {'function_id': '1'}},
 {'role': 'function',
  'content': 'что может помочь от панических атак',
  'name': 'pubmed_tools-translate_medical_query',
  'extra': {'function_id': '1'}},
 {'role': 'assistant',
  'content': '',
  'name': 'medical-assistant',
  'function_call': {'name': 'pubmed_tools-search_medical_articles',
   'arguments': '{"topic": "treatment of panic attacks", "max_results": 10}'},
  'extra': {'function_id': '1'}},
 {'role': 'function',
  'content': '{\n  "pmid": "40728738",\n  "title": "Taxonomy of anxiety disorders-a\xa0comparison of ICD‑10 and ICD‑11.",\n  "abstract": "With the introduction of the 11th\xa0revision of the World Health Organization International Statistical Classification of Diseases and Related Health Problems (ICD-11), structural and con